### Importing Libraries 

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import csv
import random
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
import random
import csv
from collections import defaultdict
import math
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from random import randint

import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

#import pylab as plt

# Set the global default size of matplotlib figures
plt.rc('figure', figsize=(10, 5))

# Size of matplotlib figures that contain subplots
fizsize_with_subplots = (10, 10)

# Size of matplotlib histogram bins
bin_size = 10
    

/opt/conda/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# FEATURE ENGINEERING

In [2]:
input_file1 = "dataset/train.csv"
input_file2 = "dataset/validation.csv"
input_file3 = "dataset/test.csv"

## comma delimited is the default, set data type to unicode
train_df = pd.read_csv(input_file1, dtype="unicode")
val_df = pd.read_csv(input_file2, dtype="unicode")
test_df = pd.read_csv(input_file3, dtype="unicode")

train_df.head(5)


,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,1,14,fdfae6789b787899f1b875de3ab8b21a,1,u_Vh1OPkFv3q5CFdR,windows_ie,180.107.112.*,80,85,...,60,1,0,5,612599432d200b093719dd1f372f7a30,300,54,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063"
1,0,2,20,372883147cdefbc495fb5082f79dde9b,1,u_VhkRL6dCOTLsq-c,windows_chrome,125.120.199.*,94,95,...,250,1,0,5,8dff45ed862a740986dbe688aafee7e5,249,42,15b749127478946f161a54dc0dad27c8,3476,"10063,10111"
2,0,5,9,2912cb5947e1fad450e2473c2f61ef75,1,u_VhkyORMrD8TP3GL,windows_ie,120.196.64.*,216,217,...,280,0,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,bebefa5efe83beee17a3d245e7c5085b,1458,10063
3,0,2,20,d7b7d61d1e50e1d27d210928d28a9b61,1,u_VhkyLRqVL2a4eWL,windows_ie,219.131.122.*,216,221,...,90,0,0,162,7184c9560e68e977187e67e45a4f3198,238,182,0f951a030abdaedd733ee8d114ce2944,3427,"10077,10111"
4,0,6,9,657d317dc4045bb79c2652abbea07530,1,u_Vh1zOJuQOtTCBgL,windows_chrome,123.190.78.*,40,48,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,34,a3becb967c015e9acb5e72f22b100ef8,3476,"10031,13042,10006,10057,13866,10063,14273,1005..."


In [3]:
# drop duplicates from all datasets
train_df.drop_duplicates()
val_df.drop_duplicates()
test_df.drop_duplicates()

,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,adexchange,...,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,keypage,advertiser,usertag
0,0,17,fe2e06dff1dfd227471fd1ca717888ac,1,u_VhkRLiMQL6dVJBn,windows_chrome,117.82.151.*,80,85,1,...,mm_10032134_3463115_11268126,300,250,1,1,0,00fccc64a1ee2809348509b7ac2a97a5,b2e35064f3549d447edbbdfb1f707c8c,3427,null
1,5,22,92ea762b7b2cfd7aee30a2e88e1f5899,1,u_Vh12CnqyP9NOQhl,windows_chrome,123.126.88.*,1,1,3,...,News_F_Width1,1000,90,0,0,80,832b91d59d0cb5731431653204a76c0e,bebefa5efe83beee17a3d245e7c5085b,1458,"10006,10083,10110"
2,2,13,617dfab714eedf4988c4aa9ad5425eca,1,u_Vh1AOk5iPt5Vebb,windows_chrome,120.14.46.*,3,8,3,...,Ent_F_bottom_Width,1000,90,0,0,20,c46090c887c257b61ab1fa11baee91d8,0f951a030abdaedd733ee8d114ce2944,3427,"10063,10006,10111"
3,4,17,7fa016aaa3db39413b482f757facc3f4,1,u_Z0p2LRsmLqsNQIl,windows_ie,123.246.112.*,40,46,1,...,mm_10024662_3445902_11178345,336,280,2,1,0,ff5123fb9333ca095034c62fdaaf51aa,361e128affece850342293213691a043,3386,"10006,13776,10110"
4,6,20,49db062ac10d19591dc7d988d730aad5,1,u_Vhk0Vwm0Olu_kGz,windows_ie,123.134.53.*,146,151,3,...,News_F_bottom_Width,1000,90,0,0,31,0cd33fcb336655841d3e1441b915748d,c8758e8658b05bc3a88af7b37bdf0575,3476,"13403,10052,10075,10063,10006,10083,10031,10110"
5,1,9,6d86a43fbda12fc89c98620f4d8aedc1,1,u_D8KBAC5wz6x,windows_chrome,27.14.200.*,275,275,3,...,News_F_Rectangle,300,250,Na,Na,80,10722,null,2821,"10684,14273,10006,10110,10131,10052,10063"
6,1,22,3b6b3f3227d42ad74971be4b9a26c189,1,u_VhT_1udeOQj9wGE,windows_chrome,183.39.8.*,216,219,2,...,3718883965,160,600,2,0,5,62f7f9a6dca2f80cc00f17dcda730bc1,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10057"
7,6,7,2cc65c3a9f97620d33d4c3ddfb368869,1,u_D9C03UFKzP1,windows_chrome,112.90.222.*,216,233,2,...,3195670606,728,90,OtherView,Na,52,7330,null,2259,"10057,10076,10083,10129,10024,13776,10120,1014..."
8,5,20,be487ced968062be378892fc87013ff9,1,u_Vh5zOkTHD4z4jdz,windows_ie,111.149.211.*,1,1,1,...,mm_29204191_2561511_9883463,728,90,0,1,0,48f2e9ba15708c0146bda5e1dd653caa,bebefa5efe83beee17a3d245e7c5085b,1458,10006
9,3,17,7f7a7e763db7ec8123ddeb8a8c90689f,1,u_VhkEP5m1P81UkVE,windows_ie,124.130.174.*,146,155,2,...,897823254,728,90,2,0,5,7184c9560e68e977187e67e45a4f3198,0f951a030abdaedd733ee8d114ce2944,3427,"10006,10024,10059,13800,10063,10111"


In [4]:
#check for wrong bidprice and pay price
train_df = train_df.drop(train_df[train_df['bidprice'] > train_df['payprice']].index)
val_df = val_df.drop(val_df[val_df['bidprice'] > val_df['payprice']].index)
train_df.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,1,14,fdfae6789b787899f1b875de3ab8b21a,1,u_Vh1OPkFv3q5CFdR,windows_ie,180.107.112.*,80,85,...,60,1,0,5,612599432d200b093719dd1f372f7a30,300,54,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063"
1,0,2,20,372883147cdefbc495fb5082f79dde9b,1,u_VhkRL6dCOTLsq-c,windows_chrome,125.120.199.*,94,95,...,250,1,0,5,8dff45ed862a740986dbe688aafee7e5,249,42,15b749127478946f161a54dc0dad27c8,3476,"10063,10111"
2,0,5,9,2912cb5947e1fad450e2473c2f61ef75,1,u_VhkyORMrD8TP3GL,windows_ie,120.196.64.*,216,217,...,280,0,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,bebefa5efe83beee17a3d245e7c5085b,1458,10063
4,0,6,9,657d317dc4045bb79c2652abbea07530,1,u_Vh1zOJuQOtTCBgL,windows_chrome,123.190.78.*,40,48,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,34,a3becb967c015e9acb5e72f22b100ef8,3476,"10031,13042,10006,10057,13866,10063,14273,1005..."
5,0,3,22,97948108bc353d0f3ef1e0142afb0343,1,u_Vh5h1p2SDvFsGMa,windows_ie,111.227.172.*,3,6,...,250,1,1,0,911b2d84826786018761e8c0b0a3a60c,227,99,d29e59bf0f7f8243858b8183f14d4412,3358,10063


In [5]:
#change categorical variables to numeric ones
# Vectorize the categorical columns: useragent
train_df_clean=train_df.drop(['bidid','logtype', 'userid','IP','adexchange','url', 'urlid','slotid','slotvisibility','slotformat', 'keypage', 'usertag'],axis=1)
train_df_clean.head()
train_df_onehot = pd.get_dummies(data=train_df_clean, columns = ['useragent','region','city'] )
train_df_onehot.head()

,click,weekday,hour,domain,slotwidth,slotheight,slotprice,creative,bidprice,payprice,...,city_90,city_91,city_92,city_93,city_94,city_95,city_96,city_97,city_98,city_99
0,0,1,14,trqRTuToMTNUjM9r5rMi,468,60,5,612599432d200b093719dd1f372f7a30,300,54,...,0,0,0,0,0,0,0,0,0,0
1,0,2,20,trqRTv14MqTR1m58uG,250,250,5,8dff45ed862a740986dbe688aafee7e5,249,42,...,0,0,0,0,0,1,0,0,0,0
2,0,5,9,trqRTJTxX5scg4,336,280,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,...,0,0,0,0,0,0,0,0,0,0
4,0,6,9,trqRTu1YP5scFsf,950,90,0,c938195f9e404b4f38c7e71bf50263e5,238,34,...,0,0,0,0,0,0,0,0,0,0
5,0,3,22,trqRTJubX5scFsf,300,250,0,911b2d84826786018761e8c0b0a3a60c,227,99,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#change categorical variables to numeric ones
val_df_clean=val_df.drop(['bidid','logtype', 'userid','IP','adexchange','url', 'urlid','slotid','slotvisibility','slotformat', 'keypage', 'usertag'],axis=1)
#val_df_clean.head()
val_df_onehot = pd.get_dummies(data=val_df_clean, columns = ['useragent','region','city'] )
val_df_onehot.head()

,click,weekday,hour,domain,slotwidth,slotheight,slotprice,creative,bidprice,payprice,...,city_90,city_91,city_92,city_93,city_94,city_95,city_96,city_97,city_98,city_99
2,0,1,13,3FKElpuEMusyJqKbuKz,1000,90,70,d01411218cc79bc49d2a4078c4093b76,241,77,...,0,0,0,0,0,0,0,0,0,0
3,0,5,18,DD1SqS9rg5scFsf,1000,90,70,c46090c887c257b61ab1fa11baee91d8,241,77,...,0,0,0,0,0,0,0,0,0,0
5,0,1,16,eS9jl55xj5scjdm4JFSyo6B_oT1A,300,250,0,e1af08818a6cd6bbba118bb54a651961,238,49,...,0,0,0,0,0,0,0,0,0,0
6,0,3,18,trqRTuMvjTN7X9KbuKz,728,90,52,b90c12ed2bd7950c6027bf9c6937c48a,300,52,...,0,0,0,0,0,0,0,0,0,0
7,0,2,0,de6d8b674cde8053f9c5c082d1c3032e,960,90,0,10717,294,86,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#change categorical variables to numeric ones
test_df_clean=test_df.drop(['bidid','logtype', 'userid','IP','adexchange','url', 'urlid','slotid','slotvisibility','slotformat', 'keypage', 'usertag'],axis=1)
#test_df_clean.head()
test_df_onehot = pd.get_dummies(data=test_df_clean, columns = ['useragent','region','city'] )
test_df_onehot.head()

,weekday,hour,domain,slotwidth,slotheight,slotprice,creative,advertiser,useragent_android_chrome,useragent_android_firefox,...,city_90,city_91,city_92,city_93,city_94,city_95,city_96,city_97,city_98,city_99
0,0,17,trqRTuxJMT27gspy,300,250,0,00fccc64a1ee2809348509b7ac2a97a5,3427,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,22,5F1RQS9rg5scFsf,1000,90,80,832b91d59d0cb5731431653204a76c0e,1458,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13,31xSTvprdN1RFt,1000,90,20,c46090c887c257b61ab1fa11baee91d8,3427,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,17,trqRTuxoGQdcFNKbuKz,336,280,0,ff5123fb9333ca095034c62fdaaf51aa,3386,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,20,5F1RQS9rg5scFsf,1000,90,31,0cd33fcb336655841d3e1441b915748d,3476,0,0,...,0,0,0,0,0,0,0,0,0,0


### Importing Dataset For Machine Learning 

In [8]:
# Setting up Dataset path
#input_file1 = "dataset/train.csv"
# Reading the first 100000 rows of the dataset
#train_df = pd.read_csv(input_file1, dtype="unicode",nrows=100000)

In [9]:
# Setting Columns
columnNames =['bidprice','payprice']
colu = ['click']

In [10]:
# Setting X Y
X = train_df[columnNames]
Y = train_df[colu]

In [11]:
# flatten y into a 1-D array
y = np.ravel(Y)

### Logistic Regression Model

In [12]:
# Setting Logistic Regression Model
model = LogisticRegression()
model = model.fit(X, y)

# check the accuracy on the training set
model.score(X, y)

0.9993104595994009

### Model Evaluation Using a Validation Set

In [13]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print (predicted)

['0' '0' '0' ..., '0' '0' '0']


In [16]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print (probs)

[[  9.99210587e-01   7.89412550e-04]
 [  9.99166543e-01   8.33456926e-04]
 [  9.99262173e-01   7.37827231e-04]
 ..., 
 [  9.99337147e-01   6.62853259e-04]
 [  9.99380781e-01   6.19218614e-04]
 [  9.99290045e-01   7.09955156e-04]]


In [17]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))

0.999300831048


In [18]:
print (metrics.confusion_matrix(y_test, predicted))
print (metrics.classification_report(y_test, predicted))

[[477376      0]
 [   334      0]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    477376
          1       0.00      0.00      0.00       334

avg / total       1.00      1.00      1.00    477710



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Model Evaluation Using Cross-Validation

In [19]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.99930921  0.99930921  0.99930921  0.99930921  0.99930921  0.99930921
  0.99930921  0.9993092   0.99931548  0.99931548]
0.999310459611


In [5]:
columnNames =['bidprice','payprice']
colu = ['click']

In [21]:
random.sample(list(train_df), 5)

['adexchange', 'usertag', 'keypage', 'domain', 'userid']

### Gradient Boosting model

In [6]:
X = train_df[columnNames]
Y = train_df[colu]
rows = random.sample(train_df.index, int(len(list(train_df)* 1)))
                     
x_train = X.ix[rows]
y_train = Y.ix[rows]
x_test = X.drop(rows)
y_test  = Y.drop(rows)

### Setting Gradient Boosting model

In [7]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 500, 'max_depth': 6,
        'learning_rate': 0.1, 'loss': 'huber','alpha':0.95}
clf = GradientBoostingRegressor(**params).fit(x_train, y_train)

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
mse = mean_squared_error(y_test, clf.predict(x_test))
r2 = r2_score(y_test, clf.predict(x_test))

print("MSE: %.4f" % mse)
print("R2: %.4f" % r2)

MSE: 0.0008
R2: -0.0008
